In [12]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler

data = pd.read_csv('1830151.csv') #loading data
data.shape 

import json

file = open('qq.json',mode='r')
all_of_it = file.read()
file.close()

data = json.loads(all_of_it)


data = pd.DataFrame(data)

df=data.copy()

df.head(1)

def con_date_time(df):
    df['clear_date']=pd.to_datetime(df['clear_date'])
    df['posting_date']=pd.to_datetime(df['posting_date'])
    df['due_in_date']=pd.to_datetime(df['due_in_date'],format='%Y-%m-%d',)
    df['document_create_date']=pd.to_datetime(df['document_create_date'],format='%Y-%m-%d', )
    df['baseline_create_date']=pd.to_datetime(df['baseline_create_date'], format='%Y-%m-%d',)
    df['business_year']=df['business_year'].astype(int)
    return None

con_date_time(df)

df['due_duration']=(df['clear_date']-df['due_in_date']).dt.days

df.dtypes

a=df
# Encoding String terms
code_business_code = {k: i for i, k in enumerate(a.business_code.unique(), 1)}
code_cust_payment_terms = {k: i for i, k in enumerate(a.cust_payment_terms.unique(), 1)}
code_cust_number = {k: i for i, k in enumerate(a.cust_number.unique(), 1)}
code_name_customer = {k: i for i, k in enumerate(a.name_customer.unique(), 1)}

def encoding(df_obj):
    df_obj.loc[:, 'code_business_code'] = df_obj.loc[:, 'business_code'].map(code_business_code)
    df_obj.loc[:, 'code_cust_payment_terms'] = df_obj.loc[:, 'cust_payment_terms'].map(code_cust_payment_terms)
    df_obj.loc[:, 'code_cust_number'] = df_obj.loc[:, 'cust_number'].map(code_cust_number)
    df_obj.loc[:, 'code_name_customer'] = df_obj.loc[:, 'name_customer'].map(code_name_customer)
    return None

# creating fetures from date
def dates_feature(df_dates):
    df_dates['posting_date_month'],df_dates['posting_date_day'],df_dates['posting_date_year'],df_dates['posting_date_quarter'],df_dates['posting_date_dayofweek'],df_dates['posting_date_week']=df_dates['posting_date'].dt.month,df_dates['posting_date'].dt.day,df_dates['posting_date'].dt.year,df_dates['posting_date'].dt.quarter,df_dates['posting_date'].dt.dayofweek,df_dates['posting_date'].dt.isocalendar().week.astype(int)
    df_dates['due_in_date_month'],df_dates['due_in_date_day'],df_dates['due_in_date_year'],df_dates['due_in_date_quarter'],df_dates['due_in_date_dayofweek'],df_dates['due_in_date_week']=df_dates['due_in_date'].dt.month,df_dates['due_in_date'].dt.day,df_dates['due_in_date'].dt.year,df_dates['due_in_date'].dt.quarter,df_dates['due_in_date'].dt.dayofweek,df_dates['due_in_date'].dt.isocalendar().week.astype(int)
    df_dates['document_create_date_month'],df_dates['document_create_date_day'],df_dates['document_create_date_year'],df_dates['document_create_date_quarter'],df_dates['document_create_date_dayofweek'],df_dates['document_create_date_week']=df_dates['document_create_date'].dt.month,df_dates['document_create_date'].dt.day,df_dates['document_create_date'].dt.year,df_dates['document_create_date'].dt.quarter,df_dates['document_create_date'].dt.dayofweek,df_dates['document_create_date'].dt.isocalendar().week.astype(int)
    df_dates['baseline_create_date_month'],df_dates['baseline_create_date_day'],df_dates['baseline_create_date_year'],df_dates['baseline_create_date_quarter'],df_dates['baseline_create_date_dayofweek'],df_dates['baseline_create_date_week']=df_dates['baseline_create_date'].dt.month,df_dates['baseline_create_date'].dt.day,df_dates['baseline_create_date'].dt.year,df_dates['baseline_create_date'].dt.quarter,df_dates['baseline_create_date'].dt.dayofweek,df_dates['baseline_create_date'].dt.isocalendar().week.astype(int)
    return None

encoding(df)
dates_feature(df)


df['document_create_is_working']=np.where(df.document_create_date_dayofweek.isin([5,6]),0,1)
df['due_in_is_working']=np.where(df.due_in_date_dayofweek.isin([5,6]),0,1)


import pickle
with open("mapper", 'rb') as f:
    mapper = pickle.load(f)

df['cust_delay_mean']=df['code_cust_number'].map(mapper[0])
df['cust_name_delay_mean']=df['code_name_customer'].map(mapper[1])
df['payment_time']=(df['due_in_date']-df['document_create_date']).dt.days

df['conv_amt']=df['total_open_amount'].copy()
df.loc[df['invoice_currency']=='CAD','conv_amt']=df.loc[df['invoice_currency']=='CAD','conv_amt']*0.78

with open("drop", 'rb') as f:
    drop = pickle.load(f)
drop[14] = "document_type"


df=df.drop(drop,axis=1, errors='ignore')

df = df.drop(["notes"],axis=1, errors='ignore')

df = df.drop(["clear_date"],axis=1, errors='ignore')

def stan(a,n):
    if n==1:
        scaler = StandardScaler()
    elif n==2:
        scaler = preprocessing.QuantileTransformer(output_distribution='normal', random_state=0)
    return scaler

with open("features", 'rb') as f:
    feature = pickle.load(f)

df = df[feature].copy()

scaler=stan(df,1)
scaler.fit(df)
Xtr=scaler.transform(df)

n_e=[]
m_d=[]
m_s=[]
m_l=[]

with open("1830151", 'rb') as f:
    qwerty = pickle.load(f)

aaa= pd.DataFrame(Xtr)

for i in aaa.columns:
    if aaa[i].isna().any():
        print(aaa[i])

for i in df.columns:
    if df[i].isna().any():
        df[i].fillna(0,inplace=True)

Xtr= np.nan_to_num(Xtr)

In [28]:
predicted = qwerty.predict(Xtr)

qwer=pd.DataFrame(np.around(predicted).astype(np.int64))

qwer[0] = qwer[0].astype(str) + ' day' 

qwer=pd.to_timedelta(qwer[0])

con_date_time(data)

Final_dates=data['due_in_date'] + qwer

final = pd.DataFrame(Final_dates,columns=['predictions'])
print(final)
# df_new = final.rename(columns={'due_in_date': 'predictions'})

# df_new

# data = pd.concat([data,df_new],axis=1)

   predictions
0   2019-01-13
1   2019-02-07
2   2019-01-01
3   2019-01-27
4   2019-01-10
..         ...
70  2019-01-28
71  2019-07-05
72  2019-05-28
73  2019-05-23
74  2021-03-18

[75 rows x 1 columns]


In [26]:
final.columns

RangeIndex(start=0, stop=1, step=1)

In [14]:
data.columns

Index(['business_code', 'cust_number', 'name_customer', 'clear_date',
       'business_year', 'doc_id', 'posting_date', 'document_create_date',
       'due_in_date', 'invoice_currency', 'document_type', 'posting_id',
       'area_business', 'total_open_amount', 'baseline_create_date',
       'cust_payment_terms', 'invoice_id', 'isOpen', 'notes', 'predictions'],
      dtype='object')

In [4]:
data['predictions']=data['predictions'].dt.strftime('%Y-%m-%d')
pred = data[['doc_id','predictions']].to_dict(orient="records")

In [6]:
pred

[{'doc_id': 1928499600, 'predictions': '2019-01-13'},
 {'doc_id': 1928509276, 'predictions': '2019-02-02'},
 {'doc_id': 1928511028, 'predictions': '2018-12-29'},
 {'doc_id': 1928511094, 'predictions': '2019-02-04'},
 {'doc_id': 1928516320, 'predictions': '2019-01-14'},
 {'doc_id': 1928516455, 'predictions': '2018-12-29'},
 {'doc_id': 1928516631, 'predictions': '2019-02-02'},
 {'doc_id': 1928517139, 'predictions': '2018-12-29'},
 {'doc_id': 1928518256, 'predictions': '2019-02-03'},
 {'doc_id': 1928521038, 'predictions': '2019-01-13'},
 {'doc_id': 1928522232, 'predictions': '2019-01-13'},
 {'doc_id': 1928526470, 'predictions': '2019-01-13'},
 {'doc_id': 1928526824, 'predictions': '2019-01-13'},
 {'doc_id': 1928526861, 'predictions': '2019-01-17'},
 {'doc_id': 1928530020, 'predictions': '2019-01-18'},
 {'doc_id': 1928530765, 'predictions': '2019-01-16'},
 {'doc_id': 1928549253, 'predictions': '2019-01-17'},
 {'doc_id': 1928549254, 'predictions': '2019-01-17'},
 {'doc_id': 1928549271, 'pre